In [ ]:
# Libs necessárias
def request(url:str = None,nome_arquivo:str = '', extensao:str='zip'):
    """AI is creating summary for request

    Args:
        url (str): Caminho online de onde possam ter redirecionamentos href.
        nome_arquivo (str): nome do arquivo que vamos buscar ou dos arquivos se tiverem mais de um com o mesmo nome.
        extensao (str): extenção do arquivo que vamos buscar se formos buscar por um download em específico.

    Returns:
        urls : Retorna uma lista com todos as urls que deram math com os parâmetros passados no crawler da url infomrada.
    """
    import re
    import urllib.request
    nome_arquivo = nome_arquivo.capitalize()
    print(nome_arquivo)
    abre_url = urllib.request.urlopen(url)
    
    le_url = abre_url.read()

    converte_leitura_url = le_url.decode("utf8")
    abre_url.close()
    
    urls = []
    #urls = [ f"{url}{i.replace('<', '')}" for i in re.findall(f"[\w\d]+.(?:zip|pdf|csv)<", converte_leitura_url) if i.startswith(nome_arquivo)]
    #urls = [f'{url}{i}.zip' for i in re.findall(f'href="(.*?).{extensao}', converte_leitura_url) if i.startswith(nome_arquivo)]
    urls = [ f"{url}{i}.{extensao}" for i in re.findall(f'href="(.*).(?:{extensao})"', converte_leitura_url) if i.startswith(nome_arquivo)]
    
    return urls
def download(url:str=None, salvar_onde:str= './' ):
    """

    Args:
        url (str): URL de do arquivo para download.
        salvar_onde (str): Pasta de destino do arquivo baixado.
    """
    from pySmartDL import SmartDL
    obj = SmartDL(url, salvar_onde)
    obj.start()
    path = obj.get_dest()

In [ ]:
links = ["https://dadosabertos.rfb.gov.br/CNPJ/", "http://200.152.38.155/CNPJ/"]
src = request(links[1])
print(src)

In [27]:
import os
class EXTRATOR_CNPJ: 
    import os
    def __init__(self, url:str = None,nome_arquivo:str = '', extensao:str='zip'): 
        self.url = url 
        self.nome_arquivo = nome_arquivo
        self.extensao = extensao 
        
    def request(self):
        import re
        import urllib.request
        abre_url = urllib.request.urlopen(self.url)
        le_url = abre_url.read()
        converte_leitura_url = le_url.decode("utf8")
        abre_url.close()
        urls = []
        urls =  [ f"{self.url}{i}.{self.extensao}" for i in re.findall(f'href="(.*).(?:{self.extensao})"', converte_leitura_url) if i.startswith(self.nome_arquivo)]
        return urls

    def download(self, url:str=None, destino:str = None):
        ### Com requests
        """
        import requests
        with requests.get(url, stream=True) as response:
            with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

        """
        ### Com urllib
        """
        import urllib.request
        arquivo = url.split('/')[-1]
        # faz o download do arquivo e salva em salvar_onde/arquivo
        urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))
        ### Com SmartDL
        """

        ### Com SmartDL
        from pySmartDL import SmartDL

        obj = SmartDL(url, destino, threads=4, progress_bar=False)
        obj.start()
        return destino

    def run(self):
        import re
        import zipfile
        from pyspark.sql import SparkSession
        urls  = self.request()
        
        # Define ou busca uma sessão do Spark
        spark = SparkSession.builder.master("local[2]").appName("OnlineReader").getOrCreate()
        for url in urls:
            
            # Diretório de execução do script
            current_dir = os.getcwd()
            
            # Pega o nome do arquivo pela url
            arquivo = url.split('/')[-1]
            
            # Define o caminho absoluto do diretório.
            salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

            # cria a pasta para armazenar o arquivo, se ela não existir
            if not os.path.exists(salvar_onde):
                os.makedirs(salvar_onde)
                
            # download do arquivo zip
            path = self.download(url, os.path.join(salvar_onde, arquivo))
            
            # descompactação do arquivo zip
            with zipfile.ZipFile(path, 'r') as zip_ref:

                # obtem o nome do primeiro arquivo dentro do zip
                nome_original_arquivo_zip = zip_ref.namelist()[0]

                # define um novo nome para o arquivo
                novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"
                # cria um dicionário com as informações de origem e destino
                arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

                # realiza a extração do arquivo zip
                zip_ref.extractall(path = f"{salvar_onde}", members=arquivos_para_extrair)
                
                # renomeia o arquivo extraído com o novo nome
                try:
                    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))
                except Exception as e:
                    print(f"deu um pequeno erro aqui {e}") 
                    os.remove(os.path.join(salvar_onde, nome_original_arquivo_zip))
                    pass
            # leitura do arquivo CSV em um dataframe Spark
            csv_file = os.path.join(salvar_onde, novo_nome_arquivo)
            dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)
            dados.show(1, vertical=True)
            os.remove(path)
            return dados

In [26]:
url = 'https://dadosabertos.rfb.gov.br/CNPJ/'
EC_SIMPLES = EXTRATOR_CNPJ(url=url, nome_arquivo='Simples', extensao='zip')
SIMPLES = EC_SIMPLES.run()

deu um pequeno erro aqui {e}
-RECORD 0-------
 _c0 | 0        
 _c1 | N        
 _c2 | 20070701 
 _c3 | 20070701 
 _c4 | N        
 _c5 | 20090701 
 _c6 | 20090701 
only showing top 1 row



DataFrame[_c0: int, _c1: string, _c2: int, _c3: int, _c4: string, _c5: int, _c6: int]

### Salva no disco e lê após isso
``` PYTHON
# Libs
import requests # SE FOR USAR REQUESTS
import urllib.request # SE FOR USAR URLLIB
from pySmartDL import SmartDL # SE FOR USAR PYSMARTDL
import zipfile
import os
from pyspark.sql import SparkSession
current_dir = os.getcwd()

# Define ou busca uma sessão do Spark
spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()

# Define a url de download dos dados
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
# Pega o nome do arquivo pela url
arquivo = url.split('/')[-1]
# Define o caminho absoluto do diretório.
salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

# cria a pasta para armazenar o arquivo, se ela não existir
if not os.path.exists(salvar_onde):
    os.makedirs(salvar_onde)

### Com requests
"""
    with requests.get(url, stream=True) as response:
        with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

"""
### Com urllib

# faz o download do arquivo e salva em salvar_onde/arquivo
urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))

### Com SmartDL
"""
    dest = os.path.join(salvar_onde, arquivo)
    obj = SmartDL(url, dest, threads=4)
    obj.start()
"""

"""
# Imprime o caminho do diretório de download
print(salvar_onde)
"""

# Descompactação do arquivo
with zipfile.ZipFile(os.path.join(salvar_onde, arquivo), 'r') as zip_ref:

    # obtem o nome do primeiro arquivo dentro do zip
    nome_original_arquivo_zip = zip_ref.namelist()[0]

    # define um novo nome para o arquivo
    novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"

    # cria um dicionário com as informações de origem e destino
    arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

    # realiza a extração do arquivo zip
    zip_ref.extractall(path = f"{salvar_onde}/", members=arquivos_para_extrair)
    
    # renomeia o arquivo extraído com o novo nome
    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))

"""
# imprime o nome do arquivo dentro do zip e o novo nome
print(f"Arquivo dentro do zip: {nome_original_arquivo_zip}")
print(f"Novo nome do arquivo: {novo_nome_arquivo}")
"""

# Define o caminho absoluto para o arquivo
csv_file = os.path.join(salvar_onde, novo_nome_arquivo)

# Lê o arquivo em um dataframe Spark
dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)

# Plota primeira linha do dataframe
dados.show(1, vertical=True)
```

In [ ]:
# Libs
import requests # SE FOR USAR REQUESTS
import urllib.request # SE FOR USAR URLLIB
from pySmartDL import SmartDL # SE FOR USAR PYSMARTDL
import zipfile
import os
from pyspark.sql import SparkSession
current_dir = os.getcwd()

# Define ou busca uma sessão do Spark
spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()

# Define a url de download dos dados
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
# Pega o nome do arquivo pela url
arquivo = url.split('/')[-1]
# Define o caminho absoluto do diretório.
salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

# cria a pasta para armazenar o arquivo, se ela não existir
if not os.path.exists(salvar_onde):
    os.makedirs(salvar_onde)

### Com requests
"""
    with requests.get(url, stream=True) as response:
        with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

"""
### Com urllib

# faz o download do arquivo e salva em salvar_onde/arquivo
urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))

### Com SmartDL
"""
    dest = os.path.join(salvar_onde, arquivo)
    obj = SmartDL(url, dest, threads=4)
    obj.start()
"""

"""
# Imprime o caminho do diretório de download
print(salvar_onde)
"""

# Descompactação do arquivo
with zipfile.ZipFile(os.path.join(salvar_onde, arquivo), 'r') as zip_ref:

    # obtem o nome do primeiro arquivo dentro do zip
    nome_original_arquivo_zip = zip_ref.namelist()[0]

    # define um novo nome para o arquivo
    novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"

    # cria um dicionário com as informações de origem e destino
    arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

    # realiza a extração do arquivo zip
    zip_ref.extractall(path = f"{salvar_onde}/", members=arquivos_para_extrair)
    
    # renomeia o arquivo extraído com o novo nome
    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))

"""
# imprime o nome do arquivo dentro do zip e o novo nome
print(f"Arquivo dentro do zip: {nome_original_arquivo_zip}")
print(f"Novo nome do arquivo: {novo_nome_arquivo}")
"""

# Define o caminho absoluto para o arquivo
csv_file = os.path.join(salvar_onde, novo_nome_arquivo)

# Lê o arquivo em um dataframe Spark
dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)

# Plota primeira linha do dataframe
dados.show(1, vertical=True)

In [28]:
dados.count()

NameError: name 'dados' is not defined